In [69]:
test_checking_long_name = pd.read_csv('../assets/train.csv')
test_checking_long_name.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent'],
      dtype='object')

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('../assets/train_cleaned.csv')
test = pd.read_csv('../assets/test_cleaned.csv')
weather = pd.read_csv('../assets/weather_cleaned.csv')
spray = pd.read_csv('../assets/spray_cleaned.csv')

In [3]:
for i in [train, test, weather]:
    i['Date'] = pd.to_datetime(i['Date'])
    i.set_index('Date', inplace=True)
    
spray['DateTime'] = pd.to_datetime(spray['DateTime'])
spray.set_index('DateTime', inplace=True)

In [4]:
for key, i in {'test':test, 'train':train, 'spray':spray, 'weather':weather}.items():
    print(key, end='')
    print(':', end='')
    display(i.head(1))
    print('\n\n')

test:

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
Date,,,,,,,,,
2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9





train:

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,PercentageWnv
Date,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,0.0





spray:

,Latitude,Longitude
DateTime,,
2011-08-29 18:56:58,42.391623,-88.089163





weather:

,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,
2007-05-04,1,66,49,58,41,50,444,1852,0,29,30.05,10.4,8,10.8


In [5]:
#play with adding the spraying info to the training set

#do the dates add up?
a_date = spray.index[0]
a_date

Timestamp('2011-08-29 18:56:58')

In [7]:
a_date.date()

datetime.date(2011, 8, 29)

In [8]:
pd.to_datetime(a_date.date())

Timestamp('2011-08-29 00:00:00')

In [9]:
train.index[0] #looks like they're in the same format

Timestamp('2007-05-29 00:00:00')

In [11]:
new_spray = spray.copy()
new_spray.index = pd.to_datetime(new_spray.index.date)

In [12]:
new_spray.index[0]

Timestamp('2011-08-29 00:00:00')

In [13]:
new_spray.shape

(14835, 2)

In [14]:
total = 0
for i in new_spray.index:
    if i in train.index:
        total+=1
print(total) #there are quite a few that occur on the same date but also quite a few that do not. I guess we should try to see if there was spraying in the past x days/weeks/months



9359


Question: 
- can we convert the long/lat into a spatial datatype? 
    - see if spraying close to an area but not in that specific area has an effect on num_mosquitos
    

In [23]:
import datetime
from datetime import timedelta


In [24]:
total = 0
n_days_ago = 7
for o in new_spray.index:
    for i in [o-timedelta(days=d) for d in range(n_days_ago+1)]:
        if i in train.index:
            total+=1
            break
print(total) #all of the sprays occured within a week of at least 1 date in the train set

14835


In [17]:
train.index[0]

Timestamp('2007-05-29 00:00:00')

In [22]:
print(a_date)
print(a_date - datetime.timedelta(days=7))

2011-08-29 18:56:58
2011-08-22 18:56:58


In [ ]:

#test to see if the long,lat pairs in spray occur in train

total = 0
for o in new_spray.index:
    for i in train.index:
        if new_spray.loc[o, 'Latitude'][0] == train.loc[i, 'Latitude'][0]:
            total += 1

print(total)

In [38]:
total = 0
lat='Latitude'
lon='Longitude'
for i in new_spray.index:
    mask_lat = train[lat]== new_spray.loc[i, lat][0]
    mask_lon = train[lon]== new_spray.loc[i, lon][0]
    temp_df = train[mask_lat]# & mask_lon]
    total += temp_df.shape[0]
    
print(total)

0


In [37]:
#mask_lat =
train[train[lat]== new_spray.loc[new_spray.index[0], lat][0]]
#mask_lon = train[lon]== new_spray.loc[i, lon][0]

,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,PercentageWnv
Date,,,,,,,,,,,,


In [40]:
print(len(set(train[lat].unique()))) #138 separate latitude traps
print(len(set(new_spray[lat].unique()))) #12887 separate latitude spraystraps
print(len(set(train[lat].unique()) - set(new_spray[lat].unique()))) # 1 single latitude overlap in the two datasets. Doesn't account for longitude.

138
12887
137


In [ ]:
#This took like 25 minutes until I interrupted it so it is not feasible for multiple columns. It did give me some insight below though

# train_lat = train.loc[train.index[0], 'Latitude'][0]
# train_lon = train.loc[train.index[0], 'Longitude'][0]

# closest_spray_lat = new_spray.loc[new_spray.index[0], 'Latitude'][0]
# closest_spray_lon = new_spray.loc[new_spray.index[0], 'Longitude'][0]

# print('initial values', "train", train_lat, train_lon, "    ", "spray", closest_spray_lat, closest_spray_lon)

# for i in new_spray.index: # each index in spray
#     for inner in range(new_spray.loc[i, 'Latitude'].shape[0]): #each row in the multiple rows that have the same date
#         if np.abs(train_lat - closest_spray_lat) > np.abs(train_lat - new_spray.loc[i, 'Latitude'][inner]):
#             #if the difference between the saved latitude and the test point is greater than the difference between the current point and the test point then the current point is closer to the test point
#             if np.abs(train_lon - closest_spray_lon) > np.abs(train_lon - new_spray.loc[i, 'Longitude'][inner]):
#                 closest_spray_lat = new_spray.loc[i, 'Latitude'][inner]
#                 closest_spray_lon = new_spray.loc[i, 'Longitude'][inner]
                    
# print()
# print('final values', "train", train_lat, train_lon, "    ", "spray", closest_spray_lat, closest_spray_lon)

In [51]:
train.loc[train.index[0], 'Latitude'][0], new_spray.loc[new_spray.index[0], 'Latitude'][0]

(41.95469, 42.3916233333333)

Oh look at that, the train lat/lon is not as precise as spray. truncate spray to be the same num significant digits as train?

are train and test lats the same length? how precise do our spraying tactics have to be? 

In [53]:
len(str(train.loc[train.index[0], 'Latitude'][0]))

8

In [55]:
float(str(new_spray.loc[new_spray.index[0], 'Latitude'][0])[:8]), new_spray.loc[new_spray.index[0], 'Latitude'][0]

(42.39162, 42.3916233333333)

In [ ]:
for i in range(new_spray.shape[0]):
    for c in [0,1]:
        new_spray.iloc[i, c] = float(str(new_spray.iloc[i, c])[:8])

In [61]:
new_spray.head(3)

,Latitude,Longitude
2011-08-29,42.391623,-88.089163
2011-08-29,42.391348,-88.089163
2011-08-29,42.391022,-88.089157


In [62]:
new_spray.head(3)

,Latitude,Longitude
2011-08-29,42.391623,-88.089163
2011-08-29,42.391348,-88.089163
2011-08-29,42.391022,-88.089157


In [63]:
train[['Latitude', 'Longitude']].head(3)

,Latitude,Longitude
Date,,
2007-05-29,41.954690,-87.800991
2007-05-29,41.954690,-87.800991
2007-05-29,41.994991,-87.769279


In [66]:
train.columns

Index(['Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'PercentageWnv'],
      dtype='object')

In [67]:
longest_lat = 0
longest_lon = 0
for i in range(train.shape[0]):
    curr_lat = len(str(train.iloc[i,6]))
    curr_lon = len(str(train.iloc[i,7]))
    
    if longest_lat < curr_lat:
        longest_lat = curr_lat
    if longest_lon < curr_lon:
        longest_lon = curr_lon

longest_lat, longest_lon

(17, 18)

In [59]:
new_spray.columns

Index(['Latitude', 'Longitude'], dtype='object')

In [68]:
len(str(new_spray.iloc[0, 0]))

16

Questions: how close are lat/long coords? could we add/subtract .00001 to each to try and include areas around the coords as opposed to just the coords themselves?

looking at the map varan has, I'm thinking of converting the lat/lon of the spray and train to 
